In [ ]:
# pip install gensim #for glove embedding

In [ ]:
import duckdb
import pandas as pd

from bs4 import BeautifulSoup


# dev_mode = True
dev_mode = False
if dev_mode:
    # DEV (user specific)
    database = "/home/heiler/development/projects/ascii/research-space/src/pipelines/ascii/ascii_dbt/ascii_pipeline.duckdb"
    prefix = "ascii_dev"
else:
    # prod
    database = "/data/raid5/data/ascii/mastered-data/ascii_pipeline.duckdb"
    prefix = "ascii"

con = duckdb.connect(
    database=database,
    read_only=True,
)

In [ ]:
%store -r df_urls

# Filter the URLs 

In [ ]:
# starting url count
len(df_urls)

In [ ]:
# first drop duplicated urls
dedup_df = df_urls.drop_duplicates(subset="src_url", keep="first")

In [ ]:
len(dedup_df)

## Step 1 filter by keywords

but only if over 50 urls for a company

In [ ]:
# keywords = ['about', 'service', 'product', 'news', 'team', 'project', 'career']

# keywords = ['about', 'service', 'product', 'news']

keywords = ["about", "service", "product", "news", "semicon", "technology"]

Now apply the keyword filter only to companies that have more than 50 urls, ensuring not too much information is lost

In [ ]:
# Identify companies with more than 50 URLs
companies_with_many_urls = (
    dedup_df.groupby("ascii_id_company")
    .filter(lambda x: len(x) > 50)["ascii_id_company"]
    .unique()
)

# Apply keyword filter to companies with more than 50 URLs
keywords_pattern = "|".join(keywords)
filtered_large_companies_df = dedup_df[
    (dedup_df["ascii_id_company"].isin(companies_with_many_urls))
    & (dedup_df["src_url"].str.contains(keywords_pattern, case=False, na=False))
]

# Get the data for companies with 50 or fewer URLs
filtered_small_companies_df = dedup_df[
    ~dedup_df["ascii_id_company"].isin(companies_with_many_urls)
]

# Concatenate the two DataFrames
filtered_df = pd.concat([filtered_large_companies_df, filtered_small_companies_df])

In [ ]:
len(filtered_df) / len(df_urls)

In [ ]:
len(filtered_df)

In [ ]:
filtered_df["ascii_id_company"].nunique()

In [ ]:
dedup_df["ascii_id_company"].nunique()

ok one had over 50 urls and now is lost. But it was a non existent company anyway.

In [ ]:
# Companies with more than 50 URLs before filtering
companies_with_many_urls_before = (
    dedup_df.groupby("ascii_id_company")
    .filter(lambda x: len(x) > 50)["ascii_id_company"]
    .unique()
)

# Companies present in the final filtered DataFrame
companies_in_final_df = filtered_df["ascii_id_company"].unique()

# Identify lost companies
lost_companies = set(companies_with_many_urls_before) - set(companies_in_final_df)

# Retrieve the original URLs for these lost companies
lost_companies_urls = dedup_df[dedup_df["ascii_id_company"].isin(lost_companies)][
    ["ascii_id_company", "src_url"]
]

print("Lost companies and their URLs:")
print(lost_companies_urls)

### if more than 200 urls (to not remove too many from few urls, keep median higher, not lose too much), remove longest 50% urls, assuming those go too much into detail and are not so relevant anymore

In [ ]:
# dont do this anymore


# def remove_longest_percent(df, percent=25, n=50):
#     # Function to remove the longest 'percent' of URLs for each company
#     def remove_urls(group):
#         n_to_remove = int(len(group) * (percent / 100))
#         if n_to_remove == 0:
#             return group
#         lengths = group['src_url'].str.len()
#         threshold_length = lengths.nlargest(n_to_remove).min()
#         return group[group['src_url'].str.len() < threshold_length]

#     # Identify companies with more than n URLs
#     companies_with_many_urls = df.groupby('ascii_id_company').filter(lambda x: len(x) > n)['ascii_id_company'].unique()

#     # Apply the removal function to companies with more than n URLs
#     filtered_large_companies_df = df[df['ascii_id_company'].isin(companies_with_many_urls)].groupby('ascii_id_company', group_keys=False).apply(remove_urls)

#     # Get the data for companies with n or fewer URLs unchanged
#     filtered_small_companies_df = df[~df['ascii_id_company'].isin(companies_with_many_urls)]

#     # Concatenate the two DataFrames
#     final_df = pd.concat([filtered_large_companies_df, filtered_small_companies_df])

#     return final_df

# # Apply the function with a custom percentage, e.g., 50%
# final_df = remove_longest_percent(filtered_df, 50,200)

In [ ]:
# len(final_df)/len(dedup_df)

In [ ]:
# final_df.iloc[:30]

In [ ]:
# len(final_df)

In [ ]:
# short statistics on the filtered urls
# Count unique src_urls per ascii_id_company
url_counts = (
    filtered_df.groupby("ascii_id_company")["src_url"]
    .nunique()
    .reset_index(name="url_count")
)

# Calculate basic statistics
min_urls = url_counts["url_count"].min()
max_urls = url_counts["url_count"].max()
avg_urls = url_counts["url_count"].mean()
med_urls = url_counts["url_count"].median()

print(f"Minimum URLs per ID: {min_urls}")
print(f"Maximum URLs per ID: {max_urls}")
print(f"Average URLs per ID: {avg_urls}")
print(f"Median URLs per ID: {med_urls}")

## filter for preferrably english urls

but if after this filter less than m urls would be left, dont apply it (ensures for instance firms that dont have this convention with /language)

In [ ]:
def filter_en_urls(df, n=50, m=30):
    # Function to filter for English URLs and revert if count falls below 'm'
    def filter_for_company(group):
        if len(group) > n:
            en_filtered = group[group["src_url"].str.contains("/en")]
            if len(en_filtered) >= m:
                return en_filtered
        return group

    # Apply the filter to each company
    filtered_df = df.groupby("ascii_id_company", group_keys=False).apply(
        filter_for_company
    )

    return filtered_df


# First, apply the English URL filter
en_filtered_df = filter_en_urls(dedup_df, 50, 50)

In [ ]:
len(en_filtered_df)

keep only 

## next step out of these urls we will do a filtering based on similarity search

In [ ]:
from gensim.downloader import load
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Load the GloVe model
glove_model = load("glove-wiki-gigaword-50")

# Define your keywords
keywords = [
    "semiconductor",
    "products",
    "services",
    "wafers",
    "chips",
    "silicon",
    "fabrication",
    "design",
    "manufacturing",
    "technology",
    "innovation",
    "electronics",
    "circuit",
    "transistor",
    "diode",
    "integrated",
    "systems",
    "development",
    "engineering",
    "industry",
]

# Convert keywords into embeddings
keyword_embeddings = np.array(
    [glove_model[word] for word in keywords if word in glove_model]
)

In [ ]:
def filter_urls_by_similarity(df, n=50, m=10):
    # Function to convert a URL into an embedding
    def url_to_embedding(url):
        # Split the URL by slashes and keep only the parts after the domain
        parts = url.split("/")[
            3:
        ]  # This excludes the 'https:', '', 'domain', and takes the rest
        # Rejoin the parts to form the path without the domain
        path = "/".join(parts)
        # Split the path into words based on common URL delimiters like slashes and hyphens
        words = [word for part in parts for word in part.split("-")]
        # Filter out words to only include meaningful ones (optional, based on your requirements)
        words = [word for word in words if word and word in glove_model]
        # Convert words into embeddings and average them to get the URL embedding
        if words:
            embeddings = np.array([glove_model[word] for word in words])
            return embeddings.mean(axis=0)
        else:
            # Return a zero vector if no meaningful words are found
            return np.zeros((glove_model.vector_size,))

    # Function to filter URLs for a single company
    def filter_for_company(group):
        if len(group) <= n:
            return group
        else:
            # Convert URLs to embeddings
            url_embeddings = np.array(
                [url_to_embedding(url) for url in group["src_url"]]
            )

            # Calculate similarity to keywords
            similarities = cosine_similarity(url_embeddings, keyword_embeddings).mean(
                axis=1
            )

            # Get indices of the top 'm' similar URLs
            top_indices = np.argsort(similarities)[-m:]

            return group.iloc[top_indices]

    # Apply the filter to each company
    filtered_df = df.groupby("ascii_id_company", group_keys=False).apply(
        filter_for_company
    )

    return filtered_df

In [ ]:
# Apply the function to your DataFrame
df = filter_urls_by_similarity(
    filtered_df, 50, 70
)  # for companies more than 50 urls, keep only the closest 70 urls

check for instance tesla which had issues before

In [ ]:
len(df[df["ascii_id_company"] == "bWyO7uUNWBS9MN2QvXHLzQ=="])

In [ ]:
for url in df["src_url"][df["ascii_id_company"] == "bWyO7uUNWBS9MN2QvXHLzQ=="]:
    print(url)

In [ ]:
for url in df["src_url"][df["ascii_id_company"] == "+xs/sbiUV1CWPJtfrGWtMw=="]:
    print(url)

In [ ]:
for url in df.iloc[:50]["src_url"]:
    print(url)

In [ ]:
len(df)

In [ ]:
df_urls_filtered = df

In [ ]:
%store df_urls_filtered